In [1]:
import numpy as np
import Mors as mors
from collections import defaultdict
import matplotlib.pyplot as plt
import scipy.integrate as integrate

In [2]:
def LxTrack(ages, stars):
    '''
    ages: 1D array of ages of stars in Myr
    stars: 1D array of Mors Star objects
   
    Returns a dictionary of arrays of Lx values over time for each star in stars
    '''

    if len(stars) > 1:
        #make a dictionary that will hold the Lx tracks
        dict = defaultdict(list)
        #make empty lists and then fill them
        for i,star in enumerate(stars):
            dict["track{0}".format(i)] = []
            for j,age in enumerate(ages):
                dict["track{0}".format(i)].append(star.Value(Age = age, Quantity='Lx'))
    
        return dict
    else:
        lx = defaultdict(list)
        star = stars[0]
        for age in ages:
            lx['track'].append(star.Value(Age = age, Quantity='Lx'))
        return lx

In [12]:
#Now do Leuv tracks
def LeuvTrack(ages, stars):
    '''
    ages: 1D array of ages of stars in Myr
    stars: 1D array of Mors Star objects
   
    Returns a dictionary of arrays of Lx values over time for each star in stars.
    '''
    if len(stars) > 1:
        #make a dictionary that will hold the Lx tracks
        dict = defaultdict(list)
        #make empty lists and then fill them
        for i,star in enumerate(stars):
            dict["track{0}".format(i)] = []
            for j,age in enumerate(ages):
                dict["track{0}".format(i)].append(star.Value(Age = age, Quantity='Leuv'))
    
        return dict
    else:
        leuv = defaultdict(list)
        star = stars[0]
        for age in ages:
            leuv["track"].append(star.Value(Age = age, Quantity='Leuv'))
        return leuv

In [4]:
def findHZ(M_star,age):
    '''
    M_star: float, stellar mass
    age: 1D array of ages in Myr

    Returns an array containing the upper and lower bounds (RecentVenus and EarlyMars) of the CHZ 
    '''
    
    HZbounds = np.empty((len(age),2))
    
    for i,t in enumerate(age):
        HZbounds[i]=(np.array([mors.aOrbHZ(Mstar=M_star,Age=t)['RecentVenus'],mors.aOrbHZ(Mstar=M_star, Age=t)['EarlyMars']]))

    #initial overlap is just the first HZ range
    overlap = HZbounds[0]
    
    for i,array in enumerate(HZbounds[1:]):
        if max(overlap[0], array[0]) <= min(overlap[1], array[1]):
            overlap = np.array((max(overlap[0], array[0]), min(overlap[1], array[1])))
    return overlap

In [2]:
#test massLost
def massLost(ages,stars,efficiency, M_pl, R_pl,chz, R_atm_frac, M_atm_frac):
    '''
    efficiency: float, ranges from 0.1-0.3
    ages: 1D array of the range of ages of the star in Myr
    M_pl: float, mass of the planet in Earth masses
    R_pl: float, radius of the planet in Earth radii
    stars: 1D array of Mors Star objects
    chz: 1D array, the distance in AU of the Continuously Habitable Zone for each mass star. Can be found by taking the mean of findHZ()

    Returns M_rem: a 1D array of the %mass remaining over time
    '''

    #Convert mass of the planet to kg
    Mp = M_pl * 5.97E24 #kg


    #Convert radius of the planet to m
    Rp = R_pl * 6.37E6 #m

    
    #Get the Lx and Leuv tracks. These are dictionaries.
    Lx_dict = LxTrack(ages, stars)
    Leuv_dict = LeuvTrack(ages, stars)

    #F=L/4pir^2
    #CHZ to cm
    #I want the order of the CHZ values to match the order of the masses of the loaded stars
    
    r = np.array(chz * 1.496E13) #cm

    
    #calculate fluxes
    flux = (np.array(list(Lx_dict.values())) + np.array(list(Leuv_dict.values())))/(4 * np.pi * r[:, np.newaxis]**2)
    
    #radius of the planetary atmosphere in meters. 
    R_xuv = Rp+Rp*R_atm_frac #meters

    #gravitational constant
    G = 6.67E-11 #si
    
    M_atm = M_atm_frac*Mp #update M_atm to be given fraction of the Mp

    #make a dictionary of the Mass Loss Rates
    Mdot = np.pi*efficiency*flux*R_xuv**2*Rp/(G*Mp)   
    
    #put Mdot in kg/year
    Mdot = Mdot*1E-3*3.15E7

    #make a dictionary of the array of remaining mass over time
    M_rem = []
    for array in Mdot:
        M_rem.append((M_atm - integrate.cumtrapz(y=array, x=ages*1e6)) / M_atm *100)
    for array in M_rem:
        array[array<0]=0
    return M_rem

In [7]:
#test massLost
def totalMassLost(ages,stars,efficiency, M_pl, R_pl,chz, R_atm_frac, M_atm_frac):
    '''
    efficiency: float, ranges from 0.1-0.3
    ages: 1D array of the range of ages of the star in Myr
    M_pl: float, mass of the planet in Earth masses
    R_pl: float, radius of the planet in Earth radii
    stars: 1D array of Mors Star objects
    chz: 1D array, the distance in AU of the Continuously Habitable Zone for each mass star. Can be found by taking the mean of findHZ()
    R_atm_frac: the radius of the planet+atmosphere (i.e. 1.1 would imply 1.1*R_pl)
    M_atm_frac: the mass of the atmosphere as a fraction of the planet's mass alone (i.e. 0.5% of the planet's mass as atmospheric mass would be 5e-3)

    Returns M_rem: a 1D array of the %mass remaining over time
    '''

    #Convert mass of the planet to kg
    Mp = M_pl * 5.97E24 #kg


    #Convert radius of the planet to m
    Rp = R_pl * 6.37E6 #m

    
    #Get the Lx and Leuv tracks. These are dictionaries.
    Lx_dict = LxTrack(ages, stars)
    Leuv_dict = LeuvTrack(ages, stars)

    #F=L/4pir^2
    #CHZ to cm
    #I want the order of the CHZ values to match the order of the masses of the loaded stars
    
    r = np.array(chz * 1.496E13) #cm

    
    #calculate fluxes
    flux = (np.array(list(Lx_dict.values())) + np.array(list(Leuv_dict.values())))/(4 * np.pi * r[:, np.newaxis]**2)
    
    #radius of the planetary atmosphere in meters. 
    R_xuv = Rp+Rp*R_atm_frac #meters

    #gravitational constant
    G = 6.67E-11 #si
    
    M_atm = M_atm_frac*Mp #update M_atm to be given fraction of the Mp

    #make a dictionary of the Mass Loss Rates
    Mdot = np.pi*efficiency*flux*R_xuv**2*Rp/(G*Mp)   
    
    #put Mdot in kg/year
    Mdot = Mdot*1E-3*3.15E7

    #make a dictionary of the array of remaining mass over time
    M_rem = []
    for array in Mdot:
        M_rem.append((integrate.cumtrapz(y=array, x=ages*1e6)))
    for array in M_rem:
        array[array<0]=0 #so the atmosphere doesn't have negative mass
    return M_rem

In [8]:
#test massLost
def Mdot(ages,stars,efficiency, M_pl, R_pl,chz, R_atm_frac, M_atm_frac):
    '''
    efficiency: float, ranges from 0.1-0.3
    ages: 1D array of the range of ages of the star in Myr
    M_pl: float, mass of the planet in Earth masses
    R_pl: float, radius of the planet in Earth radii
    stars: 1D array of Mors Star objects
    chz: 1D array, the distance in AU of the Continuously Habitable Zone for each mass star. Can be found by taking the mean of findHZ()
    R_atm_frac: the radius of the planet+atmosphere (i.e. 1.1 would imply 1.1*R_pl)
    M_atm_frac: the mass of the atmosphere as a fraction of the planet's mass alone (i.e. 0.5% of the planet's mass as atmospheric mass would be 5e-3)

    Returns M_rem: a 1D array of the %mass remaining over time
    '''

    #Convert mass of the planet to kg
    Mp = M_pl * 5.97E24 #kg


    #Convert radius of the planet to m
    Rp = R_pl * 6.37E6 #m

    
    #Get the Lx and Leuv tracks. These are dictionaries.
    Lx_dict = LxTrack(ages, stars)
    Leuv_dict = LeuvTrack(ages, stars)

    #F=L/4pir^2
    #CHZ to cm
    #I want the order of the CHZ values to match the order of the masses of the loaded stars
    
    r = np.array(chz * 1.496E13) #cm

    
    #calculate fluxes
    flux = (np.array(list(Lx_dict.values())) + np.array(list(Leuv_dict.values())))/(4 * np.pi * r[:, np.newaxis]**2)
    
    #radius of the planetary atmosphere in meters. 
    R_xuv = Rp+Rp*R_atm_frac #meters

    #gravitational constant
    G = 6.67E-11 #si
    
    M_atm = M_atm_frac*Mp #update M_atm to be given fraction of the Mp

    #make a dictionary of the Mass Loss Rates
    Mdot = np.pi*efficiency*flux*R_xuv**2*Rp/(G*Mp)   
    
    #put Mdot in kg/year
    Mdot = Mdot*1E-3*3.15E7

    return Mdot

In [5]:
def plotByMass(ages, stars, efficiency=None, Mpl=None, Rpl=None, chz=None, R_atm=None, M_atm=None,plotLxAndLeuv=True, plotMassRem=False, plotLxTrack=False, plotLeuvTrack=False, sepByMass=False):
    '''
    ages: 1D array of ages in Myr
    stars: 1D array of Mors stars objects
    efficiency, Mpl, Rpl, chz, R_atm, and M_atm are all passed to massLost(). See that function for more details.
    LxAndLeuv: plot the Lx+Leuv tracks for each mass
    MassRem: plot the %mass remaining over time for each mass
    LxTrack: plot just the Lx tracks for each mass
    LeuvTrack: plot just the Leuv tracks for each mass
    sepByMass: make one plot for each mass

    By default this function plots the Lx+Leuv tracks for all of the stars passed to it.
    It checks that the stars are of the same percentile because it wouldn't make sense to plot the tracks while varying both the rotations and masses.
    Also by default, it plots the %mass remaining over time for each star on the same plot.
    You also have the option to plot just the Lx track and just the Leuv track.
    You can also separate the plots so that not all of the masses are plotted in the same figure. (sepByMass)
    '''
    #check that the stars are all of the same percentile
    for i,star in enumerate(stars):
        if stars[0].percentile != star.percentile:
            return "Stars must be of the same rotation percentile"
    
    #make np array of the values of the LxTrack and LeuvTrack dictionaries
    LxTracks = np.array(list(LxTrack(ages,stars).values()))
    LeuvTracks = np.array(list(LeuvTrack(ages,stars).values()))
    
    
    #plots the lx + leuv tracks for each mass
    if plotLxAndLeuv:
        plt.figure()
        plt.xlim(ages[0],ages[-1])
        for i,(lxtrack,leuvtrack) in enumerate(zip(LxTracks,LeuvTracks)):
            plt.plot(ages,lxtrack+leuvtrack,label='M='+str(round(stars[i].Mstar,2)))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            if not sepByMass:
                plt.title('Lx+Leuv tracks for percentile='+str(stars[0].percentile))
                plt.legend()
                plt.xlim(ages[0],ages[-1])
            if sepByMass:
                plt.title('Lx+Leuv m='+str(stars[i].Mstar))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
    
    #plot mass remaining of all the masses
    if plotMassRem:
        if efficiency is None or Mpl is None or Rpl is None or chz is None or R_atm is None or M_atm is None:
            return "Must define the necessary parameters in order to plot the %mass remaining."
        plt.figure()
        plt.xlim(ages[0],ages[-1])
        plt.ylim(1,100)
        #mlr is not necessarily in the correct order, i.e. I currently have my list of masses not in ascending order, so if I want to plot the mass
        #from data in ascending order, i must find the index in mlr where the mass is equal
        mlr=massLost(ages, stars, efficiency, Mpl, Rpl, chz, R_atm = R_atm, M_atm = M_atm)
        for i,star in enumerate(stars):
            plt.plot(ages[1:],mlr[i], label='M='+str(round(stars[i].Mstar,2)))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            plt.ylim(1,100)
            
            if not sepByMass:
                plt.title('%Mass remaining for percentile='+str(stars[0].percentile)+'(Planet mass:'+str(Mpl)+')')
                plt.legend()
                plt.xlim(ages[0],ages[-1])
                plt.ylim(1,100)
            if sepByMass:
                plt.title('%Mass remaining for stellar mass = '+str(round(stars[i].Mstar,2)))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
                plt.ylim(1,100)

    #plot LxTrack
    if plotLxTrack:
        plt.figure()
        plt.xlim(ages[0],ages[-1])
        for i,lxtrack in enumerate(LxTracks):
            plt.plot(ages, lxtrack, label='M='+str(round(stars[i].Mstar,2)))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            if not sepByMass:
                plt.title('Lx tracks for percentile='+str(stars[0].percentile))
                plt.legend()
                plt.xlim(ages[0],ages[-1])
            if sepByMass:
                plt.title('Lx tracks m='+str(round(stars[i].Mstar,2)))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
    
    #plot LeuvTrack
    if plotLeuvTrack:
        plt.figure()
        for i,leuvtrack in enumerate(LeuvTracks):
            plt.plot(ages, leuvtrack, label='M='+str(round(stars[i].Mstar,2)))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            if not sepByMass:
                plt.title('Leuv tracks for percentile='+str(stars[0].percentile))
                plt.legend()
                plt.xlim(ages[0],ages[-1])
            if sepByMass:
                plt.title('Leuv tracks m='+str(round(stars[i].Mstar,2)))
                plt.figure()  
                plt.xlim(ages[0],ages[-1])
        

In [5]:
def plotByRot(ages, stars, efficiency=None, Mpl=None, Rpl=None, chz=None, R_atm=None, M_atm=None,plotLxAndLeuv=True, plotMassRem=False, plotLxTrack=False, plotLeuvTrack=False, sepByRot=False):
    '''
    ages: 1D array of ages in Myr
    stars: 1D array of Mors stars objects
    efficiency, Mpl, Rpl, chz, R_atm, and M_atm are all passed to massLost(). See that function for more details.
    LxAndLeuv: plot the Lx+Leuv tracks for each mass
    MassRem: plot the %mass remaining over time for each mass
    LxTrack: plot just the Lx tracks for each mass
    LeuvTrack: plot just the Leuv tracks for each mass
    sepByRot: make one plot for each rotation

    By default this function plots the Lx+Leuv tracks for all of the stars passed to it.
    It checks that the stars are of the same percentile because it wouldn't make sense to plot the tracks while varying both the rotations and masses.
    Also by default, it plots the %mass remaining over time for each star on the same plot.
    You also have the option to plot just the Lx track and just the Leuv track.
    You can also separate the plots so that not all of the masses are plotted in the same figure. (sepByMass)
    '''
    #check that the stars are all of the same percentile
    for i,star in enumerate(stars):
        if round(stars[0].Mstar,2) != round(star.Mstar,2):
            return "Stars must be of the same mass to use this function"
    
    #make np array of the values of the LxTrack and LeuvTrack dictionaries
    LxTracks = np.array(list(LxTrack(ages,stars).values()))
    LeuvTracks = np.array(list(LeuvTrack(ages,stars).values()))
    
    if len(stars) == 5:
        labels = ['Slowest','Slow','Median','Fast','Fastest']
        colors = ['red','yellow','green','cyan','blue']
        
    #plots the lx + leuv tracks for each mass
    if plotLxAndLeuv:
        plt.figure(facecolor='gainsboro',figsize=(5,5))
        plt.xlim(ages[0],ages[-1])
        
        for i,(lxtrack,leuvtrack) in enumerate(zip(LxTracks,LeuvTracks)):
            plt.plot(ages,lxtrack+leuvtrack,label=labels[i],color=colors[i])
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            tick_positions = [50,100,1000,5000]
            plt.xticks(tick_positions, [r'$50$',r'$100$',r'$1000$',r'$5000$'])
            plt.xlabel('Age (Myr)',fontsize=14)
            plt.ylabel('L$_{XUV}$ (erg/s)',fontsize=14)
            if not sepByRot:
                plt.title(r'L$_{XUV}$ Evolutionary Track: '+str(round(stars[i].Mstar,2))+'$M_\odot$',fontsize=14)
                plt.legend(title='Stellar Rotation Speed')
                plt.xlim(ages[0],ages[-1])
                tick_positions = [50,100,1000,5000]
                plt.xticks(tick_positions, [r'$50$',r'$100$',r'$1000$',r'$5000$'])
            if sepByRot:
                plt.title('Lx+Leuv %='+str(stars[i].percentile))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
    
    #plot mass remaining of all the masses
    if plotMassRem:
        if efficiency is None or Mpl is None or Rpl is None or chz is None or R_atm is None or M_atm is None:
            return "Must define the necessary parameters in order to plot the %mass remaining."
        plt.figure(facecolor='gainsboro',figsize=(5,5))
        plt.xlim(ages[0],ages[-1])
        plt.ylim(1,100)
        #mlr is not necessarily in the correct order, i.e. I currently have my list of masses not in ascending order, so if I want to plot the mass
        #from data in ascending order, i must find the index in mlr where the mass is equal
        mlr=massLost(ages, stars, efficiency, Mpl, Rpl, chz, R_atm_frac = R_atm, M_atm_frac = M_atm)
        for i,star in enumerate(stars):
            plt.plot(ages[1:],mlr[i], label=labels[i],color=colors[i])
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            plt.ylim(1,100)
            tick_positions = [50,100,1000,5000]
            plt.xticks(tick_positions, [r'$50$',r'$100$',r'$1000$',r'$5000$'])
            y_ticks = [1,10,50,100]
            plt.yticks(y_ticks, [r'$1$',r'$10$',r'$50$',r'$100$'])
            plt.xlabel('Age (Myr)',fontsize=14)
            plt.ylabel('Atmospheric Mass Remaining (%)',fontsize=14)
            
            if not sepByRot:
                plt.title('Atmospheric Mass Remaining Over Time: '+str(round(stars[i].Mstar,2))+'$M_\odot$',fontsize=14)
                plt.legend(title='Stellar Rotation Speed')
            if sepByRot:
                plt.title('%Mass remaining for percentile = '+str(stars[i].percentile))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
                plt.ylim(1,100)

    #plot LxTrack
    if plotLxTrack:
        plt.figure()
        plt.xlim(ages[0],ages[-1])
        
        for i,lxtrack in enumerate(LxTracks):
            plt.plot(ages, lxtrack, label='%='+str(stars[i].percentile))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            
            if not sepByRot:
                plt.title('Lx tracks for mass='+str(round(stars[i].Mstar,2)))
                plt.legend()
                plt.xlim(ages[0],ages[-1])
                
            if sepByRot:
                plt.title('Lx tracks %='+str(stars[i].percentile))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
                
    
    #plot LeuvTrack
    if plotLeuvTrack:
        plt.figure()
        plt.xlim(ages[0],ages[-1])
        for i,leuvtrack in enumerate(LeuvTracks):
            plt.plot(ages, leuvtrack, label='%='+str(stars[i].percentile))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlim(ages[0],ages[-1])
            
            if not sepByRot:
                plt.title('Leuv tracks for mass='+str(round(stars[i].Mstar,2)))
                plt.legend()
                plt.xlim(ages[0],ages[-1])
                
            if sepByRot:
                plt.title('Leuv tracks %='+str(stars[i].percentile))
                plt.figure()
                plt.xlim(ages[0],ages[-1])
                
        

In [21]:
def ageAtLoss(ages,stars,eff,Mpl,Rpl,chz,R_atm=1.1,M_atm=5E-3):
    '''
    ages: 1D array of ages of the stars in Myr
    stars: 1D array of Mors star objects
    eff: efficiency
    Mpl: mass of planet in earth masses
    Rpl: radius of planet in earth radii
    chz: 1D array of continuous habitable zones
    R_atm: radius of the atmosphere, default 1.1
    M_atm: mass of the atmospehre, default 5E-3
    '''
    mrem = massLost(ages,stars,eff,Mpl,Rpl,chz,R_atm,M_atm)
    aal = np.empty(len(stars))
    for i,array in enumerate(mrem):
        if len(ages[np.where(array<=1)]) != 0:
            aal[i]= ages[np.where(array<=1)][0]
        else:
            aal[i]=(float('nan'))
    
    return aal

In [8]:
def findRadius(mass):
    radii = np.empty(len(mass))
    for i,m in enumerate(mass):
        if m <= 2:
            radii[i] = m**0.28
        elif m <= 130:
            radii[i] = m**0.59 - 0.2954878
        elif m <= 26635:
            radii[i] = m**-0.04 + 16.55102861

    return radii

In [6]:
def plotContours(ages,stars,efficiency,Mpl,Rpl,chz,R_atm=1.1,M_atm=5E-3, equations=False, levels=[50], algo='mpl2014',grain=1000):
    '''
    levels should be a list of the levels at which you want the contour line equations. Default is [50].
    grain is the number of points in the smoothed masses array.
    '''
    masses = [star.Mstar for star in stars]
    mrem = [np.array(array) for array in massLost(ages,stars,efficiency,Mpl,Rpl,chz,R_atm,M_atm)]
    
    if equations:
        #plt.figure(facecolor='gainsboro')
        #plt.xscale('log')
        
        #QuadContourSet: a set of contour lines at the specified levels
        fig,ax = plt.subplots()
        contour_set = ax.contour(ages[1:], masses, mrem, levels=levels, cmap='plasma',vmin=0, vmax=100,algorithm=algo)
        plt.close(fig)
        
        contour_paths = [] #list of the paths of the contour lines
        for path in contour_set.collections[:]:
            contour_paths.append(path.get_paths())
        
        # Store (age, stellar mass) points from each contour
        contour_points = {}
        
        for i,path in enumerate(contour_paths):
            contour_points[i] = np.array(path[0].vertices)  # Extract x, y coordinates

        #store just the x values in ages_contour and just the y_values in mass_contour. This and the previous step can likely be done in one fell swoop.
        ages_contour = {}
        mass_contour = {}
        for i,array in enumerate(contour_points.values()):
            ages_contour[i] = array[:,0]
            mass_contour[i] = array[:,1]    
            
        return list(ages_contour.values()), list(mass_contour.values())
        